In [38]:
include("Hovden_VMC/MLVMC.jl")
using .MLVMC
using LinearAlgebra
import .MLVMC.nn as nn_hvdn
using BenchmarkTools

In [43]:
positions = 0.5 * randn(Float64, (numParticles, numDimensions));

In [42]:
hamiltonian = "quantumDot" # Use "quantumDot" or "calogeroSutherland" or "bosons"
interactingParticles = true

numParticles = 25
numDimensions = 1
harmonicOscillatorFrequency = 0.25
numHiddenLayers = 32

s = MLVMC.System(numParticles, numDimensions, hamiltonian, omega=harmonicOscillatorFrequency, interacting = interactingParticles)

nnet = NN(s, numHiddenLayers, numHiddenLayers, "sigmoid");

In [45]:
function kinetic_hovden(s, nnet)
    nnpls.nnAnalyticalComputeGradient!(s, nnet);
    return -0.5 * nnpls.nnAnalyticalComputeLaplacian!(nnet) / (-0.3718269050318206)
end

function kinetic_hovden_noGrad(s, nnet)
    return -0.5 * nnpls.nnAnalyticalComputeLaplacian!(nnet) / (-0.3718269050318206)
end

kinetic_hovden (generic function with 1 method)

In [48]:
psi = nn_hvdn.computePsi!(nnet, positions)

@btime kinetic_hovden(s, nnet)
@btime kinetic_hovden_clever(s, nnet)

  12.200 μs (12 allocations: 1.12 KiB)


-1.6423748735125242e-7

### Mine

In [49]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\OrbitalNeuralMethods")
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Warning: Unable to use CUDA from artifacts: The artifact at C:\Users\KarlH\.julia\artifacts\fd3b38cf5ade69a121c1ed6bc7a0a47f930ac0a1 is empty.
│ This is probably caused by a failed download. Remove the directory and try again.
└ @ CUDA.Deps C:\Users\KarlH\.julia\packages\CUDA\DfvRa\deps\bindeps.jl:84
┌ Warning: Unable to use a local CUDA installation: Could not find the nvdisasm binary.
│ If CUDA is installed, please make sure this binary is in your PATH.
└ @ CUDA.Deps C:\Users\KarlH\.julia\packages\CUDA\DfvRa\deps\bindeps.jl:94


In [50]:
n = numParticles
x = vec(position);

In [51]:
nn = NeuralNetwork([Dense(32), Sigmoid, Dense(32), Sigmoid, Dense(1)], n);

nn.layers[1].W .= nnet.variationalParameter[1]
nn.layers[1].b .= nnet.variationalParameter[2]
nn.layers[3].W .= nnet.variationalParameter[3]
nn.layers[3].b .= nnet.variationalParameter[4]
nn.layers[5].W .= nnet.variationalParameter[5]
nn.layers[5].b .= nnet.variationalParameter[6]

nn.old_output[1]= onm.model!(nn, x)[1]

-1.2220616050330717

In [53]:
@btime onm.kinetic(x, nn)

  22.400 μs (1 allocation: 16 bytes)


-4.99712259680774e-8